## 安装需要的包

In [ ]:
git clone https://github.com/ggerganov/llama.cpp
cd llama.cpp
make -j 8 llama-cli llama-quantize

## 准备您自己的 GGUF

In [ ]:
# FP16
python convert_hf_to_gguf.py ../Qwen2-1.5B-Instruct/ --outfile ../Qwen2-1.5b-instruct-f16.gguf

# FP32
python convert_hf_to_gguf.py ../Qwen2-1.5B-Instruct/ --outtype f32 --outfile ../Qwen2-1.5b-instruct-f32.gguf

In [ ]:
# 对话模式
./llama-cli -m ../Qwen2-1.5b-instruct-f16.gguf \
    -co -cnv -p "You are Qwen, created by Alibaba Cloud. You are a helpful assistant." \
    -fa -ngl 80 -n 512

# -m 或 –model:
# 显然，这是模型路径。

# -co 或 –color:
# 为输出着色以区分提示词、用户输入和生成的文本。提示文本为深黄色；用户文本为绿色；生成的文本为白色；错误文本为红色。

# -cnv 或 –conversation:
# 在对话模式下运行。程序将相应地应用聊天模板。

# -p 或 –prompt:
# 在对话模式下，它作为系统提示。

# -fa 或 –flash-attn:
# 如果程序编译时支持 GPU，则启用Flash Attention注意力实现。

# -ngl 或 –n-gpu-layers:
# 如果程序编译时支持 GPU，则将这么多层分配给 GPU 进行计算。

# -n 或 –predict:
# 要预测的token数量。

In [ ]:
# 互动模式
./llama-cli -m ../Qwen2-1.5b-instruct-f16.gguf \
    -co -sp -i -if -p "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n" \
    --in-prefix "<|im_start|>user\n" --in-suffix "<|im_end|>\n<|im_start|>assistant\n" \
    -fa -ngl 80 -n 512

# 我们在这里使用了一些新的选项：

# -sp 或 –special:
# 显示特殊token。

# -i 或 –interactive:
# 进入互动模式。你可以中断模型生成并添加新文本。

# -i 或 –interactive-first:
# 立即等待用户输入。否则，模型将立即运行并根据提示生成文本。

# -p 或 –prompt:
# 在互动模式下，这是模型续写用的上文。

# --in-prefix:
# 用户输入附加的前缀字符串。

# --in-suffix:
# 用户输入附加的后缀字符串。

In [ ]:
# 非交互模式
./llama-cli -m ../Qwen2-1.5b-instruct-f16.gguf \
    -co -sp -p "<|im_start|>system\nYou are Qwen, created by Alibaba Cloud. You are a helpful assistant.<|im_end|>\n<|im_start|>user\ngive me a short introduction to LLMs.<|im_end|>\n<|im_start|>assistant\n" \
    -fa -ngl 80 -n 512

## 无校准量化GGUF

In [ ]:
./llama-quantize ../Qwen2-1.5b-instruct-f16.gguf ../Qwen2-1.5b-instruct-q4_k_m.gguf Q4_K_M

# 这里的Q4_K_M是其中一种量化方法，还有很多，你可以查看：https://github.com/ggerganov/llama.cpp/blob/master/examples/quantize/quantize.cpp
# 但是很多证据表明，Q4_K_M 量化的最好:https://oobabooga.github.io/blog/posts/perplexities/

In [ ]:
./llama-cli -m ../Qwen2-1.5b-instruct-q4_k_m.gguf \
    -co -cnv -p "You are Qwen, created by Alibaba Cloud. You are a helpful assistant." \
    -fa -ngl 80 -n 512

到此，你可以把量化的模型发布到 Ollama 社区了。

## 使用AWQ尺度量化GGUF

为了提高量化模型的质量，一种可能的解决方案是应用[AWQ尺度](https://github.com/casper-hansen/AutoAWQ/blob/main/docs/examples.md#gguf-export)，遵循这个脚本。首先，当你使用`autoawq`运行`model.quantize()`时，记得添加`export_compatible=True`，如下所示：

In [ ]:
...
model.quantize(
    tokenizer,
    quant_config=quant_config,
    export_compatible=True
)
model.save_pretrained(quant_path)
...

上述代码实际上不会量化权重。相反，它会根据数据集调整权重，使它们“更容易”量化。

然后，当你运行`convert_hf_to_gguf.py`时，记得将模型路径替换为新模型的路径：

In [ ]:
python convert_hf_to_gguf.py <quant_path> --outfile qwen2-1.5b-instruct-f16-awq.gguf

最后，你可以像最后一个例子那样量化模型：

In [ ]:
./llama-quantize qwen2-1.5b-instruct-f16-awq.gguf ../qwen2-1.5b-instruct-q4_k_m-awq.gguf Q4_K_M

## 参考

- https://qwen.readthedocs.io/
- https://github.com/QwenLM/Qwen2.5/tree/main/docs/source/quantization
- https://mlabonne.github.io/blog/posts/Quantize_Llama_2_models_using_ggml.html